In [1]:
from bedrock_agent.utils import get_boto_session
from bedrock_agent.tool import tool
from bedrock_agent.types import ModelConfig
from bedrock_agent.agent import BedrockAgent
from dotenv import load_dotenv
import json

load_dotenv()

session = get_boto_session()

In [2]:
@tool
def get_current_weather(city: str) -> str:
    """
    Fetches the current weather for a given city
    
    :param city: The city for which the weather is to be fetched
    """
    import random
    rng_degree = random.randrange(0, 40)
    return f"The weather in {city} is {rng_degree} degrees celsius"

@tool
def write_poem(text: str) -> dict:
    """
    Generates a poem based on the input text
    
    :param text: The text to base the poem on
    """
    return json.dumps({"text": "Roses are red, violets are blue, I am a bot, and so are you"}, indent=4)

@tool
def handoff_weather_agent(request: str) -> BedrockAgent:
    """
    Forward the request to the weather agent to fetch the weather
    
    :param request: The request to forward to the weather agent
    """
    weather_tools = [get_current_weather]
    summarize_agent = BedrockAgent(
        "WeatherAgent",
        session, 
        system_message="You are a helpful agent that specializes in fetching the latest weather.",
        tools=weather_tools,
        )
    return summarize_agent

@tool
def handoff_poem_agent(request: str) -> BedrockAgent:
    """
    Forward the request to the poem agent to generate a poem
    
    :param request: The request to forward to the poem agent
    """
    poem_tools = [write_poem]
    summarize_agent = BedrockAgent(
        "PoemAgent",
        session, 
        system_message="You are a helpful agent that specializes in writing poems.",
        tools=poem_tools,
        )
    return summarize_agent

In [3]:
tools = [handoff_weather_agent, handoff_poem_agent]
model_cfg = ModelConfig(model_id="amazon.nova-pro-v1:0")
agent = BedrockAgent("ExpertAgent", session, model_cfg=model_cfg, tools=tools, verbose=False)

In [4]:
res = agent.chat("Can you write a short poem about the weather in Berlin? Please also include the current weather. Please think step by step and lay out a plan before acting.")

2024-12-19 16:53:47.285 | INFO     | bedrock_agent.agent:chat:163 - Agent: WeatherAgent | End of conversation. Returning response.
2024-12-19 16:53:49.056 | INFO     | bedrock_agent.agent:chat:163 - Agent: PoemAgent | End of conversation. Returning response.
2024-12-19 16:53:50.980 | INFO     | bedrock_agent.agent:chat:163 - Agent: ExpertAgent | End of conversation. Returning response.


In [5]:
res = agent.chat("Can you please add another story about and include the weather of Cologne?")

2024-12-19 16:53:56.832 | INFO     | bedrock_agent.agent:chat:163 - Agent: WeatherAgent | End of conversation. Returning response.
2024-12-19 16:53:57.856 | INFO     | bedrock_agent.agent:chat:163 - Agent: PoemAgent | End of conversation. Returning response.
2024-12-19 16:53:59.740 | INFO     | bedrock_agent.agent:chat:163 - Agent: ExpertAgent | End of conversation. Returning response.


In [6]:
agent.draw_trace()


--------------------------------------------------
Message # 1
--------------------------------------------------
ROLE: USER
TEXT: Can you write a short poem about the weather in Berlin? Please also include the current weather. Please think step by step and lay out a plan before acting.

--------------------------------------------------
Message # 2
--------------------------------------------------
ROLE: ASSISTANT
TEXT: Sure, let's break down the steps needed to fulfill your request:

1. **Fetch the current weather in Berlin** using the `handoff_weather_agent`.
2. **Generate a poem about the weather** using the `handoff_poem_agent`.

Here's the plan in action:


TOOL USE: handoff_weather_agent
INPUT: {'request': 'current weather in Berlin'}
TOOL USE: handoff_poem_agent
INPUT: {'request': 'Write a short poem about the weather in Berlin'}

--------------------------------------------------
Message # 3
--------------------------------------------------
ROLE: USER
TOOL RESULT: The curren

In [7]:
agent.draw_handoff_agents_traces()


Handoff Agent: WeatherAgent-1ec2

--------------------------------------------------
Message # 1
--------------------------------------------------
ROLE: USER
TEXT: current weather in Berlin

--------------------------------------------------
Message # 2
--------------------------------------------------
ROLE: ASSISTANT
TOOL USE: get_current_weather
INPUT: {'city': 'Berlin'}

--------------------------------------------------
Message # 3
--------------------------------------------------
ROLE: USER
TOOL RESULT: The weather in Berlin is 0 degrees celsius

--------------------------------------------------
Message # 4
--------------------------------------------------
ROLE: ASSISTANT
TEXT: The current weather in Berlin is 0 degrees Celsius. Please note that temperatures can vary, so it's always a good idea to check closer to your time of travel for the most accurate information.

Handoff Agent: PoemAgent-518f

--------------------------------------------------
Message # 1
--------------

In [8]:
print(f"Total tokens used: {agent.total_tokens_used} | total costs: {agent.total_costs} USD")

Total tokens used: 5691 | total costs: 0.0077736000 USD
